# This is going to be the final notebook to present all findings

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

import src.visualization as vs

In [4]:
data = pd.read_csv('data/raw/kickstarter.csv')

In [5]:
vs.nice_summary(data)

,Columns,Dtype,nunique,Non-Null Count,Missing,Missing %,Zero Count,mean,std,min,25%,50%,75%,max
0,backers_count,int64,3246,209222,-,-,18362,145.42,885.97,0.0,4.0,27.0,89.0,105857.0
1,blurb,object,180700,209214,8,-,0,-,-,-,-,-,-,-
2,category,object,169,209222,-,-,0,-,-,-,-,-,-,-
3,converted_pledged_amount,int64,31387,209222,-,-,19242,12892.9,88894.14,0.0,106.0,1537.0,6548.0,8596474.0
4,country,object,22,209222,-,-,0,-,-,-,-,-,-,-
5,created_at,int64,182158,209222,-,-,0,1456088717.33,63397109.83,1240366270.0,1413316946.75,1457895212.0,1511595013.0,1552526726.0
6,creator,object,208562,209222,-,-,0,-,-,-,-,-,-,-
7,currency,object,14,209222,-,-,0,-,-,-,-,-,-,-
8,currency_symbol,object,6,209222,-,-,0,-,-,-,-,-,-,-
9,currency_trailing_code,bool,2,209222,-,-,38111,-,-,-,-,-,-,-
